In [23]:
import pandas as pd
from datetime import datetime
import numpy as np


In [24]:
# colunas que vao para o arquivo FULL final
columns = [
    "timestamp",
    "THROTTLE",
    "BRAKE",
    "ECU_MODE_ID",
    "TORQUE_GAIN",
    "TORQUE_REF_LEFT_MOTOR",
    "TORQUE_REF_RIGHT_MOTOR",
    "LEFT_MOTOR_SPEED",
    "RIGHT_MOTOR_SPEED",
    "REAR_LEFT_WHEEL_SPEED",
    "REAR_RIGHT_WHEEL_SPEED",
    "ACCEL_LONGITUDINAL", # eixo Z
    "ACCEL_LATERAL", # eixo X
    "ACCEL_NORMAL", # eixo Y
    "VEL_ANGULAR_YAW", # eixo giro Y
    "VEL_ANGULAR_PITCH", # eixo giro X
    "VEL_ANGULAR_ROLL", # eixo giro Z
    "STACK_1_CELL_1",
    "STACK_1_CELL_2",
    "STACK_1_CELL_3",
    "STACK_1_CELL_4",
    "STACK_2_CELL_1",
    "STACK_2_CELL_2",
    "STACK_2_CELL_3",
    "STACK_2_CELL_4",
    "STACK_3_CELL_1",
    "STACK_3_CELL_2",
    "STACK_3_CELL_3",
    "STACK_3_CELL_4",
    "STACK_4_CELL_1",
    "STACK_4_CELL_2",
    "STACK_4_CELL_3",
    "STACK_4_CELL_4",
    "STACK_5_CELL_1",
    "STACK_5_CELL_2",
    "STACK_5_CELL_3",
    "STACK_5_CELL_4",
    "STACK_6_CELL_1",
    "STACK_6_CELL_2",
    "STACK_6_CELL_3",
    "STACK_6_CELL_4",
    "MAX_VOLTAGE",
    "MIN_VOLTAGE",
    "TOTAL_VOLTAGE",
    "SHUNT_CURRENT",
    "BMS_MODE_ID",
    "BMS_ERROR_ID",
    "AIR_P",
    "AIR_N"
]

In [ ]:
# timestamps de interesse
# nome: (inicio, fim)
interest_timestamps = {
    # miguelito1  
    # "miguelito1_enduro1": ("2025-12-16 11:43:00", "2025-12-16 11:43:45"), # arrancou e parou em seguida
    "miguelito1_enduro2": ("2025-12-16 11:43:00", "2025-12-16 11:43:45"), # arrancou e parou em seguida
    "miguelito1_enduro3": ("2025-12-16 11:45:15", "2025-12-16 11:47:45"), # undervoltage no final
    "miguelito1_enduro4": ("2025-12-16 11:58:15", "2025-12-16 12:01:00"), # undervoltage no final
    "miguelito1_enduro5": ("2025-12-16 12:02:50", "2025-12-16 12:04:30"), # carro rodou no final
    "miguelito1_enduro6": ("2025-12-16 12:05:50", "2025-12-16 12:11:30"), # 6 minutos andando direto
    "miguelito1_enduro7": ("2025-12-16 12:15:20", "2025-12-16 12:16:30"), # problema no pinhão da corrente ao final

    # mike1 
    "mike1_enduro1": ("2025-12-16 13:25:20", "2025-12-16 13:32:00"),
    "mike1_enduro2": ("2025-12-16 13:40:00", "2025-12-16 13:42:00"), # undervoltage no final
    "mike1_enduro3": ("2025-12-16 13:42:30", "2025-12-16 13:43:00"), # arrancada com undervoltage no final
    "mike1_enduro4": ("2025-12-16 13:44:10", "2025-12-16 13:46:15"), 

    # miguelito2
    "miguelito2_enduro1": ("2025-12-16 17:03:30", "2025-12-16 17:06:15"),
    "miguelito2_enduro2": ("2025-12-16 17:10:45", "2025-12-16 17:11:45"),
}

In [26]:
full_df = pd.DataFrame([], columns=columns)
full_df

,timestamp,THROTTLE,BRAKE,ECU_MODE_ID,TORQUE_GAIN,TORQUE_REF_LEFT_MOTOR,TORQUE_REF_RIGHT_MOTOR,LEFT_MOTOR_SPEED,RIGHT_MOTOR_SPEED,REAR_LEFT_WHEEL_SPEED,...,STACK_6_CELL_3,STACK_6_CELL_4,MAX_VOLTAGE,MIN_VOLTAGE,TOTAL_VOLTAGE,SHUNT_CURRENT,BMS_MODE_ID,BMS_ERROR_ID,AIR_P,AIR_N


In [27]:
file_names = [
    "76_DRIVER_INPUTS.csv",
    "77_ECU_MODE.csv",
    "78_TORQUE_REFERENCE.csv",
    "79_WHEEL_SPEED.csv",
    "85_LEFT_MOTOR_TRACTIVE.csv",
    "95_RIGHT_MOTOR_TRACTIVE.csv",
    "259_ACCELEROMETER.csv",
    "260_GYROSCOPE.csv",
    "261_ELETROBUILD_TEMPERATURE.csv",
    "300_STACK_1_VOLTAGE.csv",
    "301_STACK_2_VOLTAGE.csv",
    "302_STACK_3_VOLTAGE.csv",
    "303_STACK_4_VOLTAGE.csv",
    "304_STACK_5_VOLTAGE.csv",
    "305_STACK_6_VOLTAGE.csv",
    "306_ACCUMULADOR_PARAMS.csv",
    "307_BMS_PARAMS.csv",
]

folder_names = [
    "./data/dados_telemetria/oficina_151225_comp/"
]

stacks_list = [
    'STACK_1_CELL_1', 'STACK_1_CELL_2', 'STACK_1_CELL_3', 'STACK_1_CELL_4',
    'STACK_2_CELL_1', 'STACK_2_CELL_2', 'STACK_2_CELL_3', 'STACK_2_CELL_4',
    'STACK_3_CELL_1', 'STACK_3_CELL_2', 'STACK_3_CELL_3', 'STACK_3_CELL_4',
    'STACK_4_CELL_1', 'STACK_4_CELL_2', 'STACK_4_CELL_3', 'STACK_4_CELL_4',
    'STACK_5_CELL_1', 'STACK_5_CELL_2', 'STACK_5_CELL_3', 'STACK_5_CELL_4',
    'STACK_6_CELL_1', 'STACK_6_CELL_2', 'STACK_6_CELL_3', 'STACK_6_CELL_4',
]


In [28]:
full_folder = "./data/dados_telemetria/oficina_151225_comp/full"

for file_name in file_names:
    print(file_name + "...")
    dfs_list = []
    for folder in folder_names:
        dfs_list.append(pd.read_csv(f"{folder}/{file_name}"))

    combined_df = pd.concat(dfs_list, ignore_index=True)
    combined_df.to_csv(f"{full_folder}/{"FULL_" + file_name}", index=False)

76_DRIVER_INPUTS.csv...
77_ECU_MODE.csv...
78_TORQUE_REFERENCE.csv...
79_WHEEL_SPEED.csv...
85_LEFT_MOTOR_TRACTIVE.csv...
95_RIGHT_MOTOR_TRACTIVE.csv...
259_ACCELEROMETER.csv...
260_GYROSCOPE.csv...
261_ELETROBUILD_TEMPERATURE.csv...
300_STACK_1_VOLTAGE.csv...
301_STACK_2_VOLTAGE.csv...
302_STACK_3_VOLTAGE.csv...
303_STACK_4_VOLTAGE.csv...
304_STACK_5_VOLTAGE.csv...
305_STACK_6_VOLTAGE.csv...
306_ACCUMULADOR_PARAMS.csv...
307_BMS_PARAMS.csv...


In [29]:
# quais colunas precisa refatorar para colocar no arquivo final FULL.csv
# (old_name, new_name)
columns_rename_map = {
    "FULL_77_ECU_MODE.csv": [("MODE_ID", "ECU_MODE_ID"), ("TORQUE_GAIN_ID", "TORQUE_GAIN")],
    "FULL_79_WHEEL_SPEED.csv": [("REAR_LEFT_SPEED", "REAR_LEFT_WHEEL_SPEED"), ("REAR_RIGHT_SPEED", "REAR_RIGHT_WHEEL_SPEED")],
    "FULL_85_LEFT_MOTOR_TRACTIVE.csv": [("SPEED", "LEFT_MOTOR_SPEED")],
    "FULL_95_RIGHT_MOTOR_TRACTIVE.csv": [("SPEED", "RIGHT_MOTOR_SPEED")],
    "FULL_259_ACCELEROMETER.csv": [("ACCEL_Z", "ACCEL_LONGITUDINAL"), ("ACCEL_X", "ACCEL_LATERAL"), ("ACCEL_Y", "ACCEL_NORMAL")],
    "FULL_260_GYROSCOPE.csv": [("GYRO_Z", "VEL_ANGULAR_ROLL"), ("GYRO_X", "VEL_ANGULAR_PITCH"), ("GYRO_Y", "VEL_ANGULAR_YAW")],
    "FULL_300_STACK_1_VOLTAGE.csv": [
        ("CEL_1", "STACK_1_CELL_1"), ("CEL_2", "STACK_1_CELL_2"), ("CEL_3", "STACK_1_CELL_3"), ("CEL_4", "STACK_1_CELL_4")
    ],
    "FULL_301_STACK_2_VOLTAGE.csv": [
        ("CEL_1", "STACK_2_CELL_1"), ("CEL_2", "STACK_2_CELL_2"), ("CEL_3", "STACK_2_CELL_3"), ("CEL_4", "STACK_2_CELL_4")
    ],
    "FULL_302_STACK_3_VOLTAGE.csv": [
        ("CEL_1", "STACK_3_CELL_1"), ("CEL_2", "STACK_3_CELL_2"), ("CEL_3", "STACK_3_CELL_3"), ("CEL_4", "STACK_3_CELL_4")
    ],
    "FULL_303_STACK_4_VOLTAGE.csv": [
        ("CEL_1", "STACK_4_CELL_1"), ("CEL_2", "STACK_4_CELL_2"), ("CEL_3", "STACK_4_CELL_3"), ("CEL_4", "STACK_4_CELL_4")
    ],
    "FULL_304_STACK_5_VOLTAGE.csv": [
        ("CEL_1", "STACK_5_CELL_1"), ("CEL_2", "STACK_5_CELL_2"), ("CEL_3", "STACK_5_CELL_3"), ("CEL_4", "STACK_5_CELL_4")
    ],
    "FULL_305_STACK_6_VOLTAGE.csv": [
        ("CEL_1", "STACK_6_CELL_1"), ("CEL_2", "STACK_6_CELL_2"), ("CEL_3", "STACK_6_CELL_3"), ("CEL_4", "STACK_6_CELL_4")
    ],
    "FULL_306_ACCUMULADOR_PARAMS.csv": [("SHUNT_VOLTAGE", "SHUNT_CURRENT")],
    "FULL_307_BMS_PARAMS.csv": [("MODE_ID", "BMS_MODE_ID"), ("ERROR_ID", "BMS_ERROR_ID")],
}

# quais colunas vou efetivamente usar dos full individuais - nomes refatorados ja
file_columns_map = {
    'FULL_76_DRIVER_INPUTS.csv': ['THROTTLE', 'BRAKE'],
    'FULL_77_ECU_MODE.csv': ['ECU_MODE_ID', 'TORQUE_GAIN'],
    'FULL_78_TORQUE_REFERENCE.csv': ['TORQUE_REF_LEFT_MOTOR', 'TORQUE_REF_RIGHT_MOTOR'],
    'FULL_79_WHEEL_SPEED.csv': ['REAR_LEFT_WHEEL_SPEED', 'REAR_RIGHT_WHEEL_SPEED'],
    'FULL_85_LEFT_MOTOR_TRACTIVE.csv': ['LEFT_MOTOR_SPEED'],
    'FULL_95_RIGHT_MOTOR_TRACTIVE.csv': ['RIGHT_MOTOR_SPEED'],
    'FULL_259_ACCELEROMETER.csv': ['ACCEL_LONGITUDINAL', 'ACCEL_LATERAL', 'ACCEL_NORMAL'],
    'FULL_260_GYROSCOPE.csv': ['VEL_ANGULAR_ROLL', 'VEL_ANGULAR_PITCH', 'VEL_ANGULAR_YAW'],
    'FULL_300_STACK_1_VOLTAGE.csv': ['STACK_1_CELL_1', 'STACK_1_CELL_2', 'STACK_1_CELL_3', 'STACK_1_CELL_4'],
    'FULL_301_STACK_2_VOLTAGE.csv': ['STACK_2_CELL_1', 'STACK_2_CELL_2', 'STACK_2_CELL_3', 'STACK_2_CELL_4'],
    'FULL_302_STACK_3_VOLTAGE.csv': ['STACK_3_CELL_1', 'STACK_3_CELL_2', 'STACK_3_CELL_3', 'STACK_3_CELL_4'],
    'FULL_303_STACK_4_VOLTAGE.csv': ['STACK_4_CELL_1', 'STACK_4_CELL_2', 'STACK_4_CELL_3', 'STACK_4_CELL_4'],
    'FULL_304_STACK_5_VOLTAGE.csv': ['STACK_5_CELL_1', 'STACK_5_CELL_2', 'STACK_5_CELL_3', 'STACK_5_CELL_4'],
    'FULL_305_STACK_6_VOLTAGE.csv': ['STACK_6_CELL_1', 'STACK_6_CELL_2', 'STACK_6_CELL_3', 'STACK_6_CELL_4'],
    'FULL_306_ACCUMULADOR_PARAMS.csv': ['MAX_VOLTAGE', 'MIN_VOLTAGE', 'TOTAL_VOLTAGE', 'SHUNT_CURRENT'],
    'FULL_307_BMS_PARAMS.csv': ['BMS_MODE_ID', 'BMS_ERROR_ID', 'AIR_P', 'AIR_N'],
}

In [30]:
# passa pelos fulls: remove timestamps que estão fora dos ranges
def is_in_range(timestamp):
    for key in interest_timestamps:
        start_date, end_date = interest_timestamps[key]

        start_timestamp = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000
        end_timestamp = datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000

        if timestamp >= start_timestamp and timestamp <= end_timestamp:
            return True, key
        
    return False, None

full_folder = "./data/dados_telemetria/oficina_151225_comp/full"
# full_file_names = file_names + extra_file_names
full_file_names = file_names

for i in range(len(full_file_names)):
    full_file_names[i] = "FULL_" + full_file_names[i]

for file_name in full_file_names:
    print(file_name + "...")

    # verifica se existe na lista do file_columns_map
    if file_name not in file_columns_map:
        continue

    # le o arquivo atual
    df = pd.read_csv(f'{full_folder}/{file_name}')

    # monta o dicionario de dados desse arquivo
    data_row = {}

    for column in columns:
        data_row[column] = []

    # varre os timestamps
    for i in range (df.shape[0]):
        curr_timestamp = df.iloc[i, 1] 
        is_range, key = is_in_range(timestamp=curr_timestamp)
        
        if is_range:
            # pega as colunas desse arquivo
            columns_to_add = file_columns_map[file_name]

            # para cada coluna do full_df
            for col in full_df.columns:
                if col == 'timestamp': 
                    data_row['timestamp'].append(curr_timestamp)
                    continue 

                # verifica se essa coluna tem dados para adicionar
                if col in columns_to_add:
                    # se o arquivo atual esta na lista de refatoração
                    if file_name in columns_rename_map:
                        refactor_list = columns_rename_map[file_name]
                        result = next((item for item in refactor_list if item[1] == col), None)

                        if result == None:
                            # a coluna atual nao precisa de refatoração
                            data_row[col].append(df.iloc[i][col])
                            continue
                        
                        old_name, new_name = result
                        data_row[new_name].append(df.iloc[i][old_name])
                    else:
                        data_row[col].append(df.iloc[i][col])
                else:
                    # sem dados para adicionar
                    data_row[col].append(np.nan)

    # salva o dicionario na full_df
    new_rows = pd.DataFrame(data_row)
    full_df = pd.concat([full_df, new_rows], ignore_index=True)

# ordena o df por timestamp
sorted_df = full_df.sort_values(by='timestamp')

# agrupa e toma a media de timestamps duplicados
grouped_df = sorted_df.groupby('timestamp', as_index=False).mean()

# zera as medições NA da primeira linha
for j in range(grouped_df.shape[1]):
    if pd.isna(grouped_df.iloc[0, j]):
        grouped_df.iloc[0, j] = 0

# remover os NA puxando o valor do timestamp anterior
filled_df = grouped_df.fillna(method='ffill')

TRANSMISSION_RATIO = 8.89
WHEEL_RADIUS = 20.5 * 2.54 / 2 / 100
PI = 3.1415
MS_TO_KPH = 3.6
SHUNT_OFFSET = 45
ACCEL_SENSITIVITY_FACTOR = 0.122 / 1000
GYRO_SENSITIVITY_FACTOR = 70 / 1000

# formata grandezas fisica
filled_df['t'] = filled_df['timestamp'] - filled_df['timestamp'][0]
filled_df['LEFT_MOTOR_SPEED'] = filled_df['LEFT_MOTOR_SPEED'] / TRANSMISSION_RATIO * WHEEL_RADIUS * 2 * PI / 60 * MS_TO_KPH
filled_df['RIGHT_MOTOR_SPEED'] = filled_df['RIGHT_MOTOR_SPEED'] / TRANSMISSION_RATIO * WHEEL_RADIUS * 2 * PI / 60 * MS_TO_KPH
filled_df['REAR_LEFT_WHEEL_SPEED'] = filled_df['REAR_LEFT_WHEEL_SPEED'] / TRANSMISSION_RATIO * WHEEL_RADIUS * 2 * PI / 60 * MS_TO_KPH
filled_df['REAR_RIGHT_WHEEL_SPEED'] = filled_df['REAR_RIGHT_WHEEL_SPEED'] / TRANSMISSION_RATIO * WHEEL_RADIUS * 2 * PI / 60 * MS_TO_KPH
filled_df['ACCEL_LONGITUDINAL'] = filled_df['ACCEL_LONGITUDINAL'] * ACCEL_SENSITIVITY_FACTOR
filled_df['ACCEL_LATERAL'] = filled_df['ACCEL_LATERAL'] * ACCEL_SENSITIVITY_FACTOR
filled_df['ACCEL_NORMAL'] = filled_df['ACCEL_NORMAL'] * ACCEL_SENSITIVITY_FACTOR
filled_df['VEL_ANGULAR_YAW'] = filled_df['VEL_ANGULAR_YAW'] * GYRO_SENSITIVITY_FACTOR
filled_df['VEL_ANGULAR_PITCH'] = filled_df['VEL_ANGULAR_PITCH'] * GYRO_SENSITIVITY_FACTOR
filled_df['VEL_ANGULAR_ROLL'] = filled_df['VEL_ANGULAR_ROLL'] * GYRO_SENSITIVITY_FACTOR
filled_df['SHUNT_CURRENT'] = filled_df['SHUNT_CURRENT'] - SHUNT_OFFSET

# remove dados de ruidos / absurdos
for i in range(1, filled_df.shape[0]):
    if filled_df.loc[i, 'THROTTLE'] > 1500 or filled_df.loc[i, 'THROTTLE'] < 0:
        filled_df.loc[i, 'THROTTLE'] = filled_df.loc[i-1, 'THROTTLE']

    if filled_df.loc[i, 'BRAKE'] > 100 or filled_df.loc[i, 'BRAKE'] < 0:
        filled_df.loc[i, 'BRAKE'] = filled_df.loc[i-1, 'BRAKE']

    if filled_df.loc[i, 'ECU_MODE_ID'] > 10 or filled_df.loc[i, 'ECU_MODE_ID'] < 0:
        filled_df.loc[i, 'ECU_MODE_ID'] = filled_df.loc[i-1, 'ECU_MODE_ID']

    if filled_df.loc[i, 'TORQUE_GAIN'] > 1000 or filled_df.loc[i, 'TORQUE_GAIN'] < 0:
        filled_df.loc[i, 'TORQUE_GAIN'] = filled_df.loc[i-1, 'TORQUE_GAIN']

    if filled_df.loc[i, 'TORQUE_REF_LEFT_MOTOR'] > 5000 or filled_df.loc[i, 'TORQUE_REF_LEFT_MOTOR'] < 0:
        filled_df.loc[i, 'TORQUE_REF_LEFT_MOTOR'] = filled_df.loc[i-1, 'TORQUE_REF_LEFT_MOTOR']

    if filled_df.loc[i, 'TORQUE_REF_RIGHT_MOTOR'] > 5000 or filled_df.loc[i, 'TORQUE_REF_RIGHT_MOTOR'] < 0:
        filled_df.loc[i, 'TORQUE_REF_RIGHT_MOTOR'] = filled_df.loc[i-1, 'TORQUE_REF_RIGHT_MOTOR']

    if filled_df.loc[i, 'LEFT_MOTOR_SPEED'] > 100 or filled_df.loc[i, 'LEFT_MOTOR_SPEED'] < 0:
        filled_df.loc[i, 'LEFT_MOTOR_SPEED'] = filled_df.loc[i-1, 'LEFT_MOTOR_SPEED']

    if filled_df.loc[i, 'RIGHT_MOTOR_SPEED'] > 100 or filled_df.loc[i, 'RIGHT_MOTOR_SPEED'] < 0:
        filled_df.loc[i, 'RIGHT_MOTOR_SPEED'] = filled_df.loc[i-1, 'RIGHT_MOTOR_SPEED']

    if filled_df.loc[i, 'REAR_LEFT_WHEEL_SPEED'] > 100 or filled_df.loc[i, 'REAR_LEFT_WHEEL_SPEED'] < 0:
        filled_df.loc[i, 'REAR_LEFT_WHEEL_SPEED'] = filled_df.loc[i-1, 'REAR_LEFT_WHEEL_SPEED']

    if filled_df.loc[i, 'REAR_RIGHT_WHEEL_SPEED'] > 100 or filled_df.loc[i, 'REAR_RIGHT_WHEEL_SPEED'] < 0:
        filled_df.loc[i, 'REAR_RIGHT_WHEEL_SPEED'] = filled_df.loc[i-1, 'REAR_RIGHT_WHEEL_SPEED']

    if filled_df.loc[i, 'ACCEL_LONGITUDINAL'] > 10 or filled_df.loc[i, 'ACCEL_LONGITUDINAL'] < -10:
        filled_df.loc[i, 'ACCEL_LONGITUDINAL'] = filled_df.loc[i-1, 'ACCEL_LONGITUDINAL']

    if filled_df.loc[i, 'ACCEL_LATERAL'] > 10 or filled_df.loc[i, 'ACCEL_LATERAL'] < -10:
        filled_df.loc[i, 'ACCEL_LATERAL'] = filled_df.loc[i-1, 'ACCEL_LATERAL']

    if filled_df.loc[i, 'ACCEL_NORMAL'] > 10 or filled_df.loc[i, 'ACCEL_NORMAL'] < -10:
        filled_df.loc[i, 'ACCEL_NORMAL'] = filled_df.loc[i-1, 'ACCEL_NORMAL']

    if filled_df.loc[i, 'VEL_ANGULAR_YAW'] > 500 or filled_df.loc[i, 'VEL_ANGULAR_YAW'] < -500:
        filled_df.loc[i, 'VEL_ANGULAR_YAW'] = filled_df.loc[i-1, 'VEL_ANGULAR_YAW']

    if filled_df.loc[i, 'VEL_ANGULAR_ROLL'] > 500 or filled_df.loc[i, 'VEL_ANGULAR_ROLL'] < -500:
        filled_df.loc[i, 'VEL_ANGULAR_ROLL'] = filled_df.loc[i-1, 'VEL_ANGULAR_ROLL']

    if filled_df.loc[i, 'VEL_ANGULAR_PITCH'] > 500 or filled_df.loc[i, 'VEL_ANGULAR_PITCH'] < -500:
        filled_df.loc[i, 'VEL_ANGULAR_PITCH'] = filled_df.loc[i-1, 'VEL_ANGULAR_PITCH']

    for stack in stacks_list:
        if filled_df.loc[i, stack] > 10 or filled_df.loc[i, stack] < -10:
            filled_df.loc[i, stack] = filled_df.loc[i-1, stack]

    if filled_df.loc[i, 'MAX_VOLTAGE'] > 10 or filled_df.loc[i, 'MAX_VOLTAGE'] < -10:
        filled_df.loc[i, 'MAX_VOLTAGE'] = filled_df.loc[i-1, 'MAX_VOLTAGE']

    if filled_df.loc[i, 'MIN_VOLTAGE'] > 10 or filled_df.loc[i, 'MIN_VOLTAGE'] < -10:
        filled_df.loc[i, 'MIN_VOLTAGE'] = filled_df.loc[i-1, 'MIN_VOLTAGE']

    if filled_df.loc[i, 'TOTAL_VOLTAGE'] > 200 or filled_df.loc[i, 'TOTAL_VOLTAGE'] < -200:
        filled_df.loc[i, 'TOTAL_VOLTAGE'] = filled_df.loc[i-1, 'TOTAL_VOLTAGE']

    if filled_df.loc[i, 'SHUNT_CURRENT'] > 800 or filled_df.loc[i, 'SHUNT_CURRENT'] < -800:
        filled_df.loc[i, 'SHUNT_CURRENT'] = filled_df.loc[i-1, 'SHUNT_CURRENT']

    if filled_df.loc[i, 'BMS_MODE_ID'] > 50 or filled_df.loc[i, 'BMS_MODE_ID'] < 0:
        filled_df.loc[i, 'BMS_MODE_ID'] = filled_df.loc[i-1, 'BMS_MODE_ID']

    if filled_df.loc[i, 'BMS_ERROR_ID'] > 50 or filled_df.loc[i, 'BMS_ERROR_ID'] < 0:
        filled_df.loc[i, 'BMS_ERROR_ID'] = filled_df.loc[i-1, 'BMS_ERROR_ID']

    if filled_df.loc[i, 'AIR_P'] > 10 or filled_df.loc[i, 'AIR_P'] < 0:
        filled_df.loc[i, 'AIR_P'] = filled_df.loc[i-1, 'AIR_P']

    if filled_df.loc[i, 'AIR_N'] > 10 or filled_df.loc[i, 'AIR_N'] < 0:
        filled_df.loc[i, 'AIR_N'] = filled_df.loc[i-1, 'AIR_N']

# faz o split e salva em cada CSV
for key in interest_timestamps:
    folder = ""

    if "miguelito" in key: folder = "miguelito"
    if "mike" in key: folder = "mike"
    if "pedro" in key: folder = "pedro"

    start_date, end_date = interest_timestamps[key]

    start_timestamp = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000
    end_timestamp = datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000

    splitted_df = filled_df[(filled_df["timestamp"] >= start_timestamp) & (filled_df["timestamp"] < end_timestamp)]

    splitted_df.to_csv(f'{full_folder}/{folder}/{key}.csv', index=False)


# salva o df final para um CSV novo
filled_df.to_csv(f'{full_folder}/full_df.csv', index=False)

FULL_76_DRIVER_INPUTS.csv...
FULL_77_ECU_MODE.csv...
FULL_78_TORQUE_REFERENCE.csv...
FULL_79_WHEEL_SPEED.csv...
FULL_85_LEFT_MOTOR_TRACTIVE.csv...
FULL_95_RIGHT_MOTOR_TRACTIVE.csv...
FULL_259_ACCELEROMETER.csv...
FULL_260_GYROSCOPE.csv...
FULL_261_ELETROBUILD_TEMPERATURE.csv...
FULL_300_STACK_1_VOLTAGE.csv...


C:\Users\rapha\AppData\Local\Temp\ipykernel_11888\1810959298.py:74: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat([full_df, new_rows], ignore_index=True)


FULL_301_STACK_2_VOLTAGE.csv...
FULL_302_STACK_3_VOLTAGE.csv...
FULL_303_STACK_4_VOLTAGE.csv...
FULL_304_STACK_5_VOLTAGE.csv...
FULL_305_STACK_6_VOLTAGE.csv...
FULL_306_ACCUMULADOR_PARAMS.csv...
FULL_307_BMS_PARAMS.csv...


C:\Users\rapha\AppData\Local\Temp\ipykernel_11888\1810959298.py:88: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  filled_df = grouped_df.fillna(method='ffill')
C:\Users\rapha\AppData\Local\Temp\ipykernel_11888\1810959298.py:88: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  filled_df = grouped_df.fillna(method='ffill')


In [31]:
filled_df.tail(20)

,timestamp,THROTTLE,BRAKE,ECU_MODE_ID,TORQUE_GAIN,TORQUE_REF_LEFT_MOTOR,TORQUE_REF_RIGHT_MOTOR,LEFT_MOTOR_SPEED,RIGHT_MOTOR_SPEED,REAR_LEFT_WHEEL_SPEED,...,STACK_6_CELL_4,MAX_VOLTAGE,MIN_VOLTAGE,TOTAL_VOLTAGE,SHUNT_CURRENT,BMS_MODE_ID,BMS_ERROR_ID,AIR_P,AIR_N,t
15523,1765824705529,0.0,0.0,1.0,10.0,0.0,0.0,0.0,63.204736,0.0,...,3.36,3.4,3.3,0.0,-17.0,1.0,0.0,1.0,1.0,2120523
15524,1765824705562,0.0,0.0,1.0,10.0,0.0,0.0,0.0,63.204736,0.0,...,3.36,3.4,3.3,0.0,-17.0,1.0,0.0,1.0,1.0,2120556
15525,1765824705579,0.0,0.0,1.0,10.0,0.0,0.0,0.0,63.204736,0.0,...,3.36,3.4,3.3,0.0,-17.0,1.0,0.0,1.0,1.0,2120573
15526,1765824705595,0.0,0.0,1.0,10.0,0.0,0.0,0.0,63.204736,0.0,...,3.36,3.4,3.3,0.0,-17.0,1.0,0.0,1.0,1.0,2120589
15527,1765824705612,0.0,0.0,1.0,10.0,0.0,0.0,0.0,63.204736,0.0,...,3.36,3.4,3.3,0.0,-17.0,1.0,0.0,1.0,1.0,2120606
15528,1765824705628,0.0,0.0,1.0,10.0,0.0,0.0,0.0,63.204736,0.0,...,3.36,3.4,3.3,0.0,-17.0,1.0,0.0,1.0,1.0,2120622
15529,1765824705645,0.0,0.0,1.0,10.0,0.0,0.0,0.0,63.204736,0.0,...,3.36,3.4,3.3,0.0,-17.0,1.0,0.0,1.0,1.0,2120639
15530,1765824705678,0.0,0.0,1.0,10.0,0.0,0.0,0.0,63.204736,0.0,...,3.36,3.4,3.3,0.0,-17.0,1.0,0.0,1.0,1.0,2120672
15531,1765824705695,0.0,0.0,1.0,10.0,0.0,0.0,0.0,63.204736,0.0,...,3.36,3.4,3.3,0.0,-17.0,1.0,0.0,1.0,1.0,2120689
15532,1765824705728,0.0,0.0,1.0,10.0,0.0,0.0,0.0,63.204736,0.0,...,3.36,3.4,3.3,0.0,-17.0,1.0,0.0,1.0,1.0,2120722
